In [1]:
using JuMP, Gurobi, CSV, Tables

In [2]:
Students_raw = CSV.File("../Data/Name_dict.csv", header=false) |> Tables.matrix;
ALab_raw = CSV.File("../Data/Teams_ALab.csv", header=false) |> Tables.matrix;
Edge_raw = CSV.File("../Data/Teams_Edge.csv", header=false) |> Tables.matrix;
ML_raw = CSV.File("../Data/Teams_ML.csv", header=false) |> Tables.matrix;
Opt_raw = CSV.File("../Data/Teams_Opt.csv", header=false) |> Tables.matrix;

In [3]:
Students_temp = Students_raw[2:end-2,:];

In [4]:
Students = hcat(parse.(Float64, Students_temp[:,1]),Students_temp[:,2])
Students[1:5,:]

5×2 Matrix{Any}:
 1.0  "Michal Laskowski"
 2.0  "Tom Wright"
 3.0  "Jaya Ren "
 4.0  "Arushi Jain "
 5.0  "Jan Reig Torra"

In [5]:
ALab = parse.(Int64, ALab_raw[2:end,:]);
Edge = parse.(Int64, Edge_raw[2:end,:]);
ML = parse.(Int64, ML_raw[2:end,:]);
Opt = parse.(Int64, Opt_raw[2:end,:]);

In [6]:
function get_groups(mat)
    n = size(mat, 1)
    new_mat = []
    for i in 1:n
        inds = findall(x->x!=99, mat[i,:])
        push!(new_mat, Int.(mat[i,inds]))
    end
    return new_mat
end

get_groups (generic function with 1 method)

In [7]:
ALab = get_groups(ALab);
Edge = get_groups(Edge);
ML = get_groups(ML);
Opt = get_groups(Opt);

In [8]:
env = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


**Activities**: {  
    1 := sleep,  
    2 := free time,  
    3 := class,  
    4 := ALab meeting,  
    5 := Edge meeting,  
    6 := ML meeting,  
    7 := Opt meeting  
}

In [18]:
```
A := activities
S := students
C := class time periods 
```
function freetime_model(A, S, C, ALab, Edge, ML, Opt)
    
    model = Model(() -> Gurobi.Optimizer(env)) ;
    set_time_limit_sec(model, 10)
    set_optimizer_attribute(model, "OutputFlag", 1)
    set_optimizer_attribute(model, "MIPGap", 5e-3)
    
    n_A = length(A)
    n_S = size(S, 1)
    n_al = length(ALab)
    n_ed = length(Edge)
    n_ml = length(ML)
    n_opt = length(Opt)
    
    # variable for student activities over time
    @variable(model, x[1:n_S, 1:n_A, 1:7, 1:48], Bin)
    
    # variable for indicating if the group is all doing the correct meeting
    @variable(model, al[1:n_al, 1:7, 1:45], Bin)
    @variable(model, ed[1:n_ed, 1:7, 1:45], Bin)
    @variable(model, ml[1:n_ml, 1:7, 1:45], Bin)
    @variable(model, opt[1:n_opt, 1:7, 1:45], Bin)
    
    # Must do one activity at a time
#     @constraint(model, [s=1:n_S], sum(x[s,:,:,:], dims=1) .== 1)
    @constraint(model, [s=1:n_S, d=1:7, t=1:48], x[s,:,d,t] in SOS1())
    
    # at the moment, we are forcing sleep to be between 11pm and 7am (8hrs)
    @constraint(model, [s=1:n_S, d=1:7], x[s,1,d,vcat(1:14,end-1:end)].==1)
    
    # at the moment, we are forcing students to attend classes
    for day in 1:7
        @constraint(model, [s=1:n_S], x[s,3,day,C[day]].==1)
    end
    
    # ALab
    @constraint(model, sum(al, dims=(2,3)) .>= 1)
    for (i, group) in enumerate(ALab)
        @constraint(model, [d=1:7, t=1:45, s in group, t1=t:t+3], al[i,d,t] <= x[s, 4, d, t1])
    end
    
    # Edge
    @constraint(model, sum(ed, dims=(2,3)) .>= 1)
    for (i, group) in enumerate(Edge)
        @constraint(model, [d=1:7, t=1:45, s in group, t1=t:t+3], ed[i,d,t] <= x[s, 5, d, t1])
    end
    
    # ML
    @constraint(model, sum(ml, dims=(2,3)) .>= 1)
    for (i, group) in enumerate(ML)
        @constraint(model, [d=1:7, t=1:45, s in group, t1=t:t+3], ml[i,d,t] <= x[s, 6, d, t1])
    end
    
    # Opt
    @constraint(model, sum(opt, dims=(2,3)) .>= 1)
    for (i, group) in enumerate(Opt)
        @constraint(model, [d=1:7, t=1:45, s in group, t1=t:t+3], opt[i,d,t] <= x[s, 7, d, t1])
    end
    
    # objective is maximixing free time for now
    @objective(model, Max, sum(x[:,2,:,:]))
    
    optimize!(model)
 
    return value.(x), value.(al), value.(ed), value.(ml), value.(opt), model
end

freetime_model (generic function with 1 method)

In [19]:
Monday = vcat(26:28, 32:34)
Tuesday = vcat(17:20)
Wednesday = vcat(26:28, 32:34)
Thursday = vcat(17:20, 29:34)
Friday = []
Saturday = []
Sunday = []
Classes = [Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday]

7-element Vector{Vector{Any}}:
 [26, 27, 28, 32, 33, 34]
 [17, 18, 19, 20]
 [26, 27, 28, 32, 33, 34]
 [17, 18, 19, 20, 29, 30, 31, 32, 33, 34]
 []
 []
 []

In [20]:
@time x, al, ed, ml, opt, m = freetime_model([1,2,3,4,5,6,7], Students, Classes, ALab, Edge, ML, Opt)

Set parameter TimeLimit to value 10
Set parameter MIPGap to value 0.005
Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 10
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 209738 rows, 124950 columns and 444630 nonzeros
Model fingerprint: 0x9409518b
Model has 13440 SOS constraints
Variable types: 0 continuous, 124950 integer (124950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 102896 rows and 82546 columns
Presolve time: 2.58s
Presolved: 106842 rows, 42404 columns, 244012 nonzeros
Variable types: 0 continuous, 42404 integer (42404 binary)
Found heuristic solution: objective 7280.0000000

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Root relaxation preso

([1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0;;; 1.0 0.0 … 0.0

In [24]:
argmax(ed[10,:,:])

CartesianIndex(5, 32)

In [26]:
Edge[10]

2-element Vector{Int64}:
 12
 15

In [23]:
x[Edge[2],5,6,34:37]

4×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

In [47]:
sum(x[:,:,:,:], dims=(1,2))

1×1×7×48 Array{Float64, 4}:
[:, :, 1, 1] =
 40.0

[:, :, 2, 1] =
 40.0

[:, :, 3, 1] =
 40.0

[:, :, 4, 1] =
 40.0

[:, :, 5, 1] =
 40.0

[:, :, 6, 1] =
 40.0

[:, :, 7, 1] =
 40.0

[:, :, 1, 2] =
 40.0

[:, :, 2, 2] =
 40.0

[:, :, 3, 2] =
 40.0

[:, :, 4, 2] =
 40.0

[:, :, 5, 2] =
 40.0

[:, :, 6, 2] =
 40.0

[:, :, 7, 2] =
 40.0

[:, :, 1, 3] =
 40.0

[:, :, 2, 3] =
 40.0

[:, :, 3, 3] =
 40.0

[:, :, 4, 3] =
 40.0

[:, :, 5, 3] =
 40.0

[:, :, 6, 3] =
 40.0

[:, :, 7, 3] =
 40.0

;;;; … 

[:, :, 1, 46] =
 6.0

[:, :, 2, 46] =
 3.0

[:, :, 3, 46] =
 8.0

[:, :, 4, 46] =
 13.0

[:, :, 5, 46] =
 1.0

[:, :, 6, 46] =
 5.0

[:, :, 7, 46] =
 4.0

[:, :, 1, 47] =
 40.0

[:, :, 2, 47] =
 40.0

[:, :, 3, 47] =
 40.0

[:, :, 4, 47] =
 40.0

[:, :, 5, 47] =
 40.0

[:, :, 6, 47] =
 40.0

[:, :, 7, 47] =
 40.0

[:, :, 1, 48] =
 40.0

[:, :, 2, 48] =
 40.0

[:, :, 3, 48] =
 40.0

[:, :, 4, 48] =
 40.0

[:, :, 5, 48] =
 40.0

[:, :, 6, 48] =
 40.0

[:, :, 7, 48] =
 40.0

In [29]:
ALab[1:2]

2-element Vector{Any}:
 [1, 32, 36, 6]
 [2, 28]

In [32]:
argmax(al[2,:,:])

CartesianIndex(6, 41)

In [33]:
x[ALab[2],4,6,41:45]

2×5 Matrix{Float64}:
 1.0  1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0  0.0